In [ ]:
%pip install Dlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten
import keras as keras
from skimage import feature
import re
import seaborn as sns
from sklearn.cluster import KMeans
import sys
import os
import shutil
import dlib

##  K-Means para separa as pastas e agrupamentos

In [ ]:
h_kmeans = [file for file in glob.glob('Projeto/Fotos/Q/*/*.jpg')]
h_kmeans = [file.replace('\\', '/') for file in h_kmeans]

In [ ]:
labels = []
dataset_k = {}
brilho = 50

for i, file in enumerate(h_kmeans):
    
    img = cv2.imread(file) ### ler a imagem
        
    img_512 = cv2.resize(img,(512, 512),interpolation=cv2.INTER_AREA)
        
    img = cv2.cvtColor(img_512, cv2.COLOR_BGR2RGB)
    
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)
    img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    plt.imshow(img_bright);
    plt.show()
    img_gray = cv2.cvtColor(img_bright, cv2.COLOR_RGB2GRAY)
    
    #ataset_k.append(img_gray)
    dataset_k[file] = img_gray
       
    label = file.split(sep='/')[3]
    print(label)    
    
    name = file.split(sep='/')[-1]
    print(name)    
    
    cv2.imwrite(f'Projeto/Fotos/Q_head/' + name + '.png', img_gray)
    labels.append(label)

In [ ]:
list(dataset_k.values())[4].shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(list(dataset_k.values())[4], cmap="gray")

In [ ]:
list(dataset_k.keys())[4]

In [ ]:
img_arr = np.asarray(list(dataset_k.values()))
img_k = img_arr.reshape(img_arr.shape[0], -1)
img_k.shape

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
print(img_k[1])

In [ ]:
K = 36
k_means = KMeans(n_clusters=K)

In [ ]:
k_means.fit(img_k)

## Predict

In [ ]:
agrupamento = k_means.predict(img_k)
agrupamento

In [ ]:
agrupamento = [20, 20,  9,  0,  3,  3, 27, 24, 10, 10, 25, 32,  2, 34,  9,  9, 29,
               35,  6, 15,  3,  3, 31,  7,  4,  4, 13, 13, 26,  8,  8, 18,  1, 16,
               21,  5, 22, 23, 11, 11,  2, 12, 28, 14, 33, 17, 33, 19, 30,  2]

## Criando e movendo para as pastas

In [ ]:
dict_k = {k:f'Projeto/Fotos/Q/cluster_{k}'for k in np.unique(agrupamento)}

In [ ]:
for caminho in dict_k.values():
    if not os.path.exists(caminho):
        os.makedirs(caminho)


In [ ]:
dict_k

In [ ]:
dict_k[20]

In [ ]:
for i, cluster in enumerate(agrupamento):
    arquivo = list(dataset_k.keys())[i]
    
    print(f"arquivo: {arquivo},cluster: {cluster}, caminho da pasta cluster: {dict_k[cluster]}\n")

In [ ]:
for i, cluster in enumerate(agrupamento):
    arquivo = list(dataset_k.keys())[i]
    
    shutil.move(arquivo, dict_k[cluster])

### D-lib para identificar as pastas separadas com cada imagem e coloca-las junto

In [ ]:
files = [file for file in glob.glob('Projeto/Fotos/H/*/*.jpg')]

files = [file.replace('\\', '/') for file in files]

In [ ]:
img = cv2.imread(files[4])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_resize = cv2.resize(img,(500, 500),interpolation=cv2.INTER_AREA)

img_gray = cv2.cvtColor(img_resize, cv2.COLOR_RGB2GRAY)

plt.imshow(img_gray)

In [ ]:
detector_face_cnn = dlib.cnn_face_detection_model_v1('../Visao Computacional/Weights/mmod_human_face_detector.dat')

deteccoes = detector_face_cnn(img_gray, 1),

len(deteccoes)

In [ ]:
while len(deteccoes) == 0:
    
    height, width = img_gray.shape[:2]
    
    center = (width/2, height/2)
    
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)
        
    img_gray = cv2.warpAffine(src=img_gray, M=rotate_matrix, dsize=(width, height))
        
    deteccoes = detector_face_cnn(img_gray, 1)

    print(deteccoes)
        
    #plt.figure(figsize=(15,15))
    
    plt.imshow(img)
    
    plt.show()

In [ ]:
for face in deteccoes:
    l, t, r, b, c = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom(), face.confidence
    print(c)
    cv2.rectangle(img, (l,t), (r,b), (255,255,0), 2)
    plt.figure(figsize=(10,10))
    plt.imshow(img)

# Teste com CNN

In [ ]:
files = [file for file in glob.glob('Projeto/Fotos/H/*/*.jpg')]

files = [file.replace('\\', '/') for file in files]

detector_face_cnn = dlib.cnn_face_detection_model_v1('../Visao Computacional/Weights/mmod_human_face_detector.dat')

brilho = 50

face_corte = {}

for i, file in enumerate(files):
    
    img = cv2.imread(file) ### ler a imagem

    imagem_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_resize = cv2.resize(imagem_rgb,(600, 600),interpolation=cv2.INTER_AREA)

    img_hsv = cv2.cvtColor(img_resize, cv2.COLOR_RGB2HSV)
    
    img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)

    img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    
    img_gray = cv2.cvtColor(img_bright, cv2.COLOR_RGB2GRAY)

    deteccoes = detector_face_cnn(img_gray, 1)

    print(len(deteccoes))

    plt.imshow(img_gray)

    plt.show()

    while len(deteccoes) == 0:
    
        height, width = img_gray.shape[:2]
        
        center = (width/2, height/2)
        
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)
            
        img_gray = cv2.warpAffine(src=img_gray, M=rotate_matrix, dsize=(width, height))

        deteccoes = detector_face_cnn(img_gray, 1)

        print(deteccoes)
    
        plt.imshow(img_gray)
    
        plt.show()

    for face in deteccoes:

        l, t, r, b, c = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom(), face.confidence
        
        print(c)
        
        cv2.rectangle(img_gray, (l,t), (r,b), (255,255,0), 2)

        plt.figure(figsize=(10,10))
        
        plt.imshow(img_gray)

        plt.show()

        corte = img_gray[t:b, l:r]

        face_corte[file] = corte

    

# Teste com Hog_face

In [ ]:
files = [file for file in glob.glob('Projeto/Fotos/H/*/*.jpg')]

files = [file.replace('\\', '/') for file in files]

labels = []

dataset_h = {}

detector_face_hog = dlib.get_frontal_face_detector()

brilho = 15

for i, file in enumerate(files):
    
    img = cv2.imread(file) ### ler a imagem
    
    imagem_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_resize = cv2.resize(imagem_rgb,(300, 300),interpolation=cv2.INTER_AREA)

    #img_hsv = cv2.cvtColor(img_resize, cv2.COLOR_RGB2HSV)
    
    #img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)

    #img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    
    #img_gray = cv2.cvtColor(img_bright, cv2.COLOR_RGB2GRAY)
    
    deteccoes = detector_face_hog(img_resize, 4)
    
    print(len(deteccoes))

    plt.imshow(img_resize)

    plt.show()

    while len(deteccoes) == 0:
    
        height, width = img_resize.shape[:2]
        
        center = (width/2, height/2)
        
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)
            
        img_resize = cv2.warpAffine(src=img_resize, M=rotate_matrix, dsize=(width, height))

        deteccoes = detector_face_hog(img_resize, 4)

        print(deteccoes)
    
        plt.imshow(img_resize)
    
        plt.show()

        
    for face in deteccoes:
        
        l, t, r, b = face.left(), face.top(), face.right(), face.bottom()

        cv2.rectangle(img_resize, (l, t), (r, b), (50,255,255), 2)
        
        print(l, t, r, b)
        
        plt.imshow(img_resize)
        plt.show()

        plt.imshow(img_resize[t:b, l:r])
        plt.show()
    
    #img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
    #label = file.split(sep='/')[3]
    #print(label)    
    
    #name = file.split(sep='/')[-1]
    #print(name)
    
    #cv2.imwrite(f'Projeto/' + label + '/' + name + '.png', img_gray)
    #labels.append(label)

In [ ]:

files = [file for file in glob.glob('Projeto/Fotos/H/*/*.jpg')]

files = [file.replace('\\', '/') for file in files]

detector_face_hog = dlib.get_frontal_face_detector()

In [ ]:
brilho = 10
img = cv2.imread(files[1])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_resize = cv2.resize(img,(500, 500),interpolation=cv2.INTER_AREA)


img_hsv = cv2.cvtColor(img_resize, cv2.COLOR_RGB2HSV)

img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)

img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

#deteccoes = detector_face_hog(img_gray, 4) # escala

#print(len(deteccoes))

plt.figure(figsize=(10,10))
plt.imshow(img_bright)

In [ ]:
files = [file for file in glob.glob('Projeto/Fotos/H/01/*.jpg')]

files = [file.replace('\\', '/') for file in files]

detector_facial = cv2.CascadeClassifier('../Visao Computacional/Cascades/haarcascade_frontalface_default.xml')

brilho = 70

face_corte = {}

for i, file in enumerate(files):

    img = cv2.imread(file)

    img_resize = cv2.resize(img,(800, 800),interpolation=cv2.INTER_AREA)

    img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)

    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        
    img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)

    img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

    img_gray = cv2.cvtColor(img_bright, cv2.COLOR_RGB2GRAY)

    deteccoes = detector_facial.detectMultiScale(img_gray, scaleFactor = 1.3, minSize = (100,100))

    while len(deteccoes) == 0:
    
        height, width = img_gray.shape[:2]
        
        center = (width/2, height/2)
        
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)
            
        img_gray = cv2.warpAffine(src=img_gray, M=rotate_matrix, dsize=(width, height))

        deteccoes = detector_facial.detectMultiScale(img_gray, scaleFactor = 1.3, minSize = (90,90))

        print(deteccoes)
    
        plt.imshow(img_gray)
    
        plt.show()

    for x, y, w, h in deteccoes:

        #print(x, y, w, h)
        
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (180,25,55), 2)

        recorte_face = img_rgb[y:y+h, x:x+w]

        plt.imshow(img_rgb)
        
        plt.show()

        plt.imshow(recorte_face)
    
        plt.show()

        face_corte[file] = recorte_face

In [ ]:
files = [file for file in glob.glob('Projeto/Fotos/H/01/*.jpg')]

files = [file.replace('\\', '/') for file in files]

detector_facial = cv2.CascadeClassifier('../Visao Computacional/Cascades/haarcascade_frontalface_default.xml')

brilho = 40


In [ ]:
img = cv2.imread(files[0])

img_resize = cv2.resize(img,(600, 600),interpolation=cv2.INTER_AREA)

img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)

img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    
img_hsv[:,:,2] = cv2.add(img_hsv[:,:,2], brilho)

img_bright = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

img_gray = cv2.cvtColor(img_bright, cv2.COLOR_RGB2GRAY)

deteccoes = detector_facial.detectMultiScale(img_gray, scaleFactor = 1.2, minSize = (60,60))

In [ ]:
while len(deteccoes) == 0:
    
        height, width = img_gray.shape[:2]
        
        center = (width/2, height/2)
        
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)
            
        img_gray = cv2.warpAffine(src=img_gray, M=rotate_matrix, dsize=(width, height))

        img_rgb = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)

        deteccoes = detector_facial.detectMultiScale(img_gray, scaleFactor = 1.2, minSize = (60,60))

        print(deteccoes)
    
        plt.imshow(img_rgb)
    
        plt.show()

In [ ]:
plt.imshow(img_bright)


In [ ]:
for x, y, w, h in deteccoes:

        #print(x, y, w, h)

        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (180,25,55), 2)

        recorte_face = img_rgb[y:y+h, x:x+w]

        plt.imshow(img_rgb)

        plt.show()

        plt.imshow(recorte_face)

        plt.show()

        face_corte[file] = recorte_face

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(list(dataset_h.values())[1000])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset_h, labels, test_size=0.33, stratify=labels, random_state=42)

X_train = np.asarray(X_train)

X_test = np.asarray(X_test)

print('tamanho do dataset de treino:', X_train.shape)

print('tamanho do dataset de teste:', X_test.shape)

In [ ]:
plt.imshow(X_train[4]);
print(y_train[4])

In [ ]:
X_train_rf = X_train.reshape(X_train.shape[0], -1)

X_test_rf = X_test.reshape(X_test.shape[0], -1)

print(X_train_rf.shape)
print(X_test_rf.shape)

In [ ]:
clf = RandomForestClassifier()

clf.fit(X_train_rf, y_train) 

y_pred_rf = clf.predict(X_test_rf)

y_pred_train = clf.predict(X_train_rf)

In [ ]:
print(classification_report(y_train, y_pred_train ))
print(classification_report(y_test, y_pred_rf))

In [ ]:
confusion_matrix(y_test, y_pred_rf)

In [ ]:

plt.figure(figsize=[10, 10])

fig = ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf)

plt.show(fig)

In [ ]:
img_256 = X_train[4]

#img_512 = cv2.resize(teste,(512, 512),interpolation=cv2.INTER_AREA)

plt.figure(figsize=[10, 10])

plt.imshow(img_256);

In [ ]:
print(labels)

In [ ]:
plt.imshow(labels[0])

clusterização

dlib

template matching